In [1]:

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152

from importlib.machinery import SourceFileLoader
import uproot as up
import numpy as np
import time
import iminuit
import sys
import tensorflow as tf
tf.get_logger().setLevel('INFO')

time1 = time.time()

sys.path.append('/software/pc24403/tfpcbpggsz/core')
sys.path.append('/software/pc24403/tfpcbpggsz/func')
sys.path.append('/software/pc24403/tfpcbpggsz/amp_ampgen_test')
from amp import *
from core import *
from tfmassshape import *


2024-07-03 14:08:52.181341: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-03 14:09:02.940193: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


D0ToKSpipi2018 (Apr 30 2024) ==> Initialization !


2024-07-03 14:09:18.190254: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2024-07-03 14:09:18.190357: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:134] retrieving CUDA diagnostic information for host: hm01.dice.priv
2024-07-03 14:09:18.190367: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:141] hostname: hm01.dice.priv
2024-07-03 14:09:18.190557: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:165] libcuda reported version is: 550.54.15
2024-07-03 14:09:18.190654: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:169] kernel reported version is: 525.85.12
2024-07-03 14:09:18.190663: E external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:251] kernel version 525.85.12 does not match DSO version 550.54.15 -- cannot find working devices in this configuration


In [2]:
index=1
update=True

In [3]:
def get_mass(p1,p2):
    return ((p1[:,0]+p2[:,0])**2 - (p1[:,1]+p2[:,1])**2 - (p1[:,2]+p2[:,2])**2 - (p1[:,3]+p2[:,3])**2)


def get_p4(decay="b2dpi", cut='', index=index):

    file_name = ''
    branch_names = []
    if cut == 'int':
        file_name = f'/software/pc24403/PCBPGGSZ/Int/weighted_{decay}.root:DalitzEventList'
        branch_names = ["_1_K0S0_E", "_1_K0S0_Px", "_1_K0S0_Py", "_1_K0S0_Pz",
                         "_2_pi#_E", "_2_pi#_Px", "_2_pi#_Py", "_2_pi#_Pz",
                         "_3_pi~_E", "_3_pi~_Px", "_3_pi~_Py", "_3_pi~_Pz"]
    
    elif decay.split('_')[0] == 'b2dk' or decay.split('_')[0] == 'b2dpi':
        branch_names = ["_1_K0S0_E", "_1_K0S0_Px", "_1_K0S0_Py", "_1_K0S0_Pz",
                         "_2_pi#_E", "_2_pi#_Px", "_2_pi#_Py", "_2_pi#_Pz",
                         "_3_pi~_E", "_3_pi~_Px", "_3_pi~_Py", "_3_pi~_Pz"]
        if cut == 'p':
            file_name = f'/software/pc24403/PCBPGGSZ/outputs/toy/1x/{decay}_test_{index}.root:Bplus_DalitzEventList'
        else:
            file_name = f'/software/pc24403/PCBPGGSZ/outputs/toy/1x/{decay}_test_{index}.root:Bminus_DalitzEventList'

    tree = up.open(file_name)
  # Load the branches as arrays
    charge = '(Bac_ID>0)'
    if cut == 'm':
        charge = '(Bac_ID<0)'
       
    array = tree.arrays(branch_names, charge)
    _p1 = np.asarray([array["_1_K0S0_E"], array["_1_K0S0_Px"], array["_1_K0S0_Py"], array["_1_K0S0_Pz"]])
    _p2 = np.asarray([array["_2_pi#_E"], array["_2_pi#_Px"], array["_2_pi#_Py"], array["_2_pi#_Pz"]])
    _p3 = np.asarray([array["_3_pi~_E"], array["_3_pi~_Px"], array["_3_pi~_Py"], array["_3_pi~_Pz"]])
    
    # convert 4*1000 into a vectot<double>
    p1 = np.transpose(_p1)
    p2 = np.transpose(_p2)
    p3 = np.transpose(_p3)

    p1bar = np.hstack((p1[:, :1], np.negative(p1[:, 1:])))
    p2bar = np.hstack((p2[:, :1], np.negative(p2[:, 1:])))
    p3bar = np.hstack((p3[:, :1], np.negative(p3[:, 1:])))



    return p1, p2, p3, p1bar, p2bar, p3bar

def load_int_amp(args):
    p1, p2, p3 = args

    return Kspipi.AMP(p1.tolist(), p2.tolist(), p3.tolist())

def getAmp(decay='b2dpi', cut='int', index=index):

    start_time = time.time()
    p1, p2, p3, p1bar, p2bar, p3bar = get_p4(decay=decay, cut=cut, index=index)
    amplitude = []
    amplitudeBar = []

    p1_np = np.array(p1)
    p2_np = np.array(p2)
    p3_np = np.array(p3)
    p1bar_np = np.array(p1bar)
    p2bar_np = np.array(p2bar)
    p3bar_np = np.array(p3bar)

    data = [(p1_np[i], p2_np[i], p3_np[i]) for i in range(len(p1_np))]
    amplitude = [load_int_amp(args) for args in data]
    data_bar = [(p1bar_np[i], p3bar_np[i], p2bar_np[i]) for i in range(len(p1bar_np))]
    amplitudeBar = [load_int_amp(args) for args in data_bar]
    
    end_time = time.time()
    print(f'Amplitude for {decay} loaded in {end_time-start_time} seconds')
    amplitude = np.array(amplitude)
    amplitudeBar = np.negative(np.array(amplitudeBar))

    return amplitude, amplitudeBar
    
def get_p4_v2(decay="b2dpi", cut='', index=index, comp='sig'):

    file_name = ''
    branch_names = []
    if cut == 'int':
        file_name = f'/software/pc24403/PCBPGGSZ/Int/weighted_{decay}.root:DalitzEventList'
        branch_names = ["_1_K0S0_E", "_1_K0S0_Px", "_1_K0S0_Py", "_1_K0S0_Pz",
                         "_2_pi#_E", "_2_pi#_Px", "_2_pi#_Py", "_2_pi#_Pz",
                         "_3_pi~_E", "_3_pi~_Px", "_3_pi~_Py", "_3_pi~_Pz"]
    
    elif decay.split('_')[0] == 'b2dk' or decay.split('_')[0] == 'b2dpi':
        branch_names = ["_1_K0S0_E", "_1_K0S0_Px", "_1_K0S0_Py", "_1_K0S0_Pz",
                         "_2_pi#_E", "_2_pi#_Px", "_2_pi#_Py", "_2_pi#_Pz",
                         "_3_pi~_E", "_3_pi~_Px", "_3_pi~_Py", "_3_pi~_Pz", "B_M"]
        if cut == 'p':
            file_name = f'/software/pc24403/PCBPGGSZ/outputs/toy/1x/{decay}_test_{index}.root:Bplus_DalitzEventList'

        else:
            file_name = f'/software/pc24403/PCBPGGSZ/outputs/toy/1x/{decay}_test_{index}.root:Bminus_DalitzEventList'

    tree = up.open(file_name)
  # Load the branches as arrays
    charge = '(Bac_ID>0)'
    if cut == 'm':
        charge = '(Bac_ID<0)'
    
    array = tree.arrays(branch_names, charge)
       

    _p1 = np.asarray([array["_1_K0S0_E"], array["_1_K0S0_Px"], array["_1_K0S0_Py"], array["_1_K0S0_Pz"]])
    _p2 = np.asarray([array["_2_pi#_E"], array["_2_pi#_Px"], array["_2_pi#_Py"], array["_2_pi#_Pz"]])
    _p3 = np.asarray([array["_3_pi~_E"], array["_3_pi~_Px"], array["_3_pi~_Py"], array["_3_pi~_Pz"]])
    
    # convert 4*1000 into a vectot<double>
    p1 = np.transpose(_p1)
    p2 = np.transpose(_p2)
    p3 = np.transpose(_p3)

    p1bar = np.hstack((p1[:, :1], np.negative(p1[:, 1:])))
    p2bar = np.hstack((p2[:, :1], np.negative(p2[:, 1:])))
    p3bar = np.hstack((p3[:, :1], np.negative(p3[:, 1:])))

    B_M = np.zeros(len(p1))
    if cut != 'int':
        
        B_M = np.asarray([array["B_M"]])


    return p1, p2, p3, p1bar, p2bar, p3bar, B_M


def getMass(decay='b2dpi', cut='int', index=index):

    p1, p2, p3, p1bar, p2bar, p3bar = get_p4(decay=decay, cut=cut, index=index)


    p1_np = np.array(p1)
    p2_np = np.array(p2)
    p3_np = np.array(p3)

    s12 = get_mass(p1_np, p2_np)
    s13 = get_mass(p1_np, p3_np)

    return s12, s13

def getMass_v2(decay='b2dpi', cut='int', comp='sig', index=index):

    p1, p2, p3, p1bar, p2bar, p3bar, B_M = get_p4_v2(decay=decay, cut=cut, comp=comp, index=index)


    p1_np = np.array(p1)
    p2_np = np.array(p2)
    p3_np = np.array(p3)

    s12 = get_mass(p1_np, p2_np)
    s13 = get_mass(p1_np, p3_np)

    return s12, s13, B_M



def scanner(m, scan_range=1, size=50, n_var=6):
    pts = {}
    for i in range(0, n_var):
        for j in range(i + 1, n_var):
            a = m.parameters[i]
            b = m.parameters[j]
            a_err = m.errors[i]
            b_err = m.errors[j]
            a_val = m.values[i]
            b_val = m.values[j]
            x0, x1 = i, j  # Assign x0 and x1 for key naming
            pts[f"{x0}_{x1}"] = m.contour(
                a, b,
                bound=[
                    [a_val - scan_range * a_err, a_val + scan_range * a_err],
                    [b_val - scan_range * b_err, b_val + scan_range * b_err]
                ],
                size=size
            )
    return pts


config_mass_shape_output = SourceFileLoader('config_mass_shape_output', '/software/pc24403/PCBPGGSZ/outputs/toy/mass_fit/config/%s'%('config_mass_shape_output_1.py')).load_module()

varDict = config_mass_shape_output.getconfig()

pdfs_data = {}
s12_data = {}
s13_data = {}
Bu_M = {}
mass_pdfs = {}
for decay in ['b2dk_LL', 'b2dk_DD']:
    for charge in ['p', 'm']:
        new_decay = decay + '_'+ charge
        print('--- INFO: Building function for %s...'%new_decay)
        s12_data[new_decay], s13_data[new_decay], Bu_M[new_decay] = getMass_v2(decay, charge, index)
        Bu_M[new_decay] = tf.cast(Bu_M[new_decay], tf.float64)
        #pdfs_data[new_decay] = preparePdf_data(Bu_M[new_decay], varDict, decay)

comps = ['sig', 'misid', 'comb', 'low', 'low_misID', 'low_Bs2DKPi']
for decay in ['b2dk_LL', 'b2dk_DD']:
    for charge in ['p', 'm']:
        new_decay = decay + '_'+ charge
        mass_pdfs[new_decay] = {}
        for comp in comps:
            if decay.split('_')[0] == 'b2dpi' and (comp == 'low_Bs2DKPi' or comp == 'low_misID'): continue
            mass_pdfs[new_decay][comp] = []
time2 = time.time()

eff={}
eff_misid={}
eff_low={}
eff_comb_a={}
eff_comb_abar={}
print('INFO: Calculate efficiency...')
for decay in ['b2dk_LL', 'b2dk_DD']:
    for charge in ['p', 'm']:
        new_decay = decay + '_'+ charge
        eff[new_decay] = eff_fun(dalitz_transform(s12_data[new_decay], s13_data[new_decay]), charge, decay)
        eff_low[new_decay] = eff_fun(dalitz_transform(s12_data[new_decay], s13_data[new_decay]), charge, decay.replace('dk', 'dpi'))
        eff_comb_a[new_decay] = eff_fun(dalitz_transform(s12_data[new_decay], s13_data[new_decay]), 'm', decay.replace('dk', 'dpi'))
        eff_comb_abar[new_decay] = eff_fun(dalitz_transform(s12_data[new_decay], s13_data[new_decay]), 'p', decay.replace('dk', 'dpi'))
        if decay.split('_')[0] == 'b2dpi':
            eff_misid[new_decay] = eff_fun(dalitz_transform(s12_data[new_decay], s13_data[new_decay]), charge, decay.replace('dpi', 'dk'))
        elif decay.split('_')[0] == 'b2dk':
            eff_misid[new_decay] = eff_fun(dalitz_transform(s12_data[new_decay], s13_data[new_decay]), charge, decay.replace('dk', 'dpi'))       

print('INFO: mass Function loaded...')



frac_bkg = {}
if update == True:
    print('INFO: Updating Log...')
    comp_tag = {'sig': 1, 'comb': 2, 'misid': 3, 'low': 4, 'low_misID': 5, 'low_Bs2DKPi': 6}
    for decay in ['DK_KsPiPi_LL', 'DK_KsPiPi_DD']:
        bkg=0
        sig=0
        new_decay = decay
        if decay.split('_')[0] == 'DK':
            new_decay = 'b2dk_'+decay.split('_')[2]
        elif decay.split('_')[0] == 'DPi':
            new_decay = 'b2dpi_'+decay.split('_')[2]
        file_name = f'/software/pc24403/PCBPGGSZ/outputs/toy/1x/{new_decay}_test_{index}.root:Bplus_DalitzEventList'
        tree = up.open(file_name)
        for comp in comp_tag:
            array = tree.arrays('tagmode','(B_M>5080) & (tagmode==%d)'%comp_tag[comp])
            varDict[f'n_{comp}_{decay}'] = tf.cast(2*len(array), tf.float64)

        for charge in ['p', 'm']:
            Bcharge = 'Bplus'
            charge_flag = '(Bac_ID>0) '
            if charge == 'm':
                Bcharge = 'Bminus'
                charge_flag = '(Bac_ID<0) '
            file_name = f'/software/pc24403/PCBPGGSZ/outputs/toy/1x/{new_decay}_test_{index}.root:{Bcharge}_DalitzEventList'
            tree = up.open(file_name)
            for comp in comp_tag:
                array = tree.arrays('tagmode',f'(B_M>5080) & (tagmode==%d) & {charge_flag}'%comp_tag[comp])
                varDict[f'n_{comp}_{decay}_{charge}'] = tf.cast(len(array), tf.float64)
                print(f'n_{comp}_{decay}_{charge} = {len(array)}')
                if comp == 'sig':
                    sig += len(array)
                else:
                    bkg += len(array)



            frac_bkg[f'{new_decay}_{charge}'] = bkg/(sig+bkg)

print(f'frac_bkg = {frac_bkg}')

print_yields = True
sig_yield = {}
Bdecays = ['b2dk']
Types = ['LL', 'DD']
for bdecay in Bdecays:
    for Type in Types:
        decay = ''

        if bdecay == 'b2dk':
            decay = 'DK_KsPiPi_%s'%Type
        elif bdecay == 'b2dpi':
            decay = 'DPi_KsPiPi_%s'%Type
        #print yields
        if print_yields == False:
            print('Yields:')
            print('Sig: %.2f'%varDict['n_sig_%s'%decay])
            print('MisID: %.2f'%varDict['n_misid_%s'%decay])
            print('Low: %.2f'%varDict['n_low_%s'%decay])
            print('Comb: %.2f'%varDict['n_comb_%s'%decay])
            if bdecay == 'b2dk':
                print('Low MisID: %.2f'%varDict['n_low_misID_%s'%decay])
                print('Low Bs2DKPi: %.2f'%varDict['n_low_Bs2DKPi_%s'%decay])
                sum_yields = varDict['n_sig_%s'%decay] + varDict['n_misid_%s'%decay] + varDict['n_low_%s'%decay] + varDict['n_comb_%s'%decay] + varDict['n_low_misID_%s'%decay] + varDict['n_low_Bs2DKPi_%s'%decay]
            else:
                sum_yields = varDict['n_sig_%s'%decay] + varDict['n_misid_%s'%decay] + varDict['n_low_%s'%decay] + varDict['n_comb_%s'%decay]
            print('Sum: %.2f'%sum_yields)

frac_DD_dic = {}

for decay in ['b2dk_LL', 'b2dk_DD']:
    for charge in ['p', 'm']:
        Bcharge = 'Bplus'
        charge_flag = '(Bac_ID>0) '
        if charge == 'm':
            Bcharge = 'Bminus'
            charge_flag = '(Bac_ID<0) '
        file_name = f'/software/pc24403/PCBPGGSZ/outputs/toy/1x/{decay}_comb_{index}.root:{Bcharge}_DalitzEventList'
        tree = up.open(file_name)
        array_flat = tree.arrays(f'tagmode',f'(B_M>5080) & (flav==9) & {charge_flag}')
        array_dd = tree.arrays(f'tagmode',f'(B_M>5080) & (flav!=9) & {charge_flag}')
        frac_DD_dic[decay+'_'+charge] = tf.cast(len(array_dd)/(len(array_dd)+len(array_flat)), tf.float64)
        
print(f'frac_DD = {frac_DD_dic}')



logpath = '/dice/users/pc24403/BPGGSZ/sim_fit_frac'
if os.path.exists(logpath) == False:
    os.mkdir(logpath)


print('INFO: Initialize the amplitudes...')



print('INFO: Loading amplitudes...')


time3 = time.time()

--- INFO: Building function for b2dk_LL_p...
--- INFO: Building function for b2dk_LL_m...
--- INFO: Building function for b2dk_DD_p...
--- INFO: Building function for b2dk_DD_m...
INFO: Calculate efficiency...
INFO: mass Function loaded...
INFO: Updating Log...
n_sig_DK_KsPiPi_LL_p = 2438
n_comb_DK_KsPiPi_LL_p = 904
n_misid_DK_KsPiPi_LL_p = 0
n_low_DK_KsPiPi_LL_p = 0
n_low_misID_DK_KsPiPi_LL_p = 0
n_low_Bs2DKPi_DK_KsPiPi_LL_p = 0
n_sig_DK_KsPiPi_LL_m = 2438
n_comb_DK_KsPiPi_LL_m = 904
n_misid_DK_KsPiPi_LL_m = 0
n_low_DK_KsPiPi_LL_m = 0
n_low_misID_DK_KsPiPi_LL_m = 0
n_low_Bs2DKPi_DK_KsPiPi_LL_m = 0
n_sig_DK_KsPiPi_DD_p = 5417
n_comb_DK_KsPiPi_DD_p = 1943
n_misid_DK_KsPiPi_DD_p = 0
n_low_DK_KsPiPi_DD_p = 0
n_low_misID_DK_KsPiPi_DD_p = 0
n_low_Bs2DKPi_DK_KsPiPi_DD_p = 0
n_sig_DK_KsPiPi_DD_m = 5417
n_comb_DK_KsPiPi_DD_m = 1943
n_misid_DK_KsPiPi_DD_m = 0
n_low_DK_KsPiPi_DD_m = 0
n_low_misID_DK_KsPiPi_DD_m = 0
n_low_Bs2DKPi_DK_KsPiPi_DD_m = 0
frac_bkg = {'b2dk_LL_p': 0.2704967085577499, 'b2

In [4]:
mc_path = '/shared/scratch/pc24403/amp_ampgen'
mc_noeff_path = '/shared/scratch/pc24403/amp_ampgen_noeff'
amp_Data={}
amp_MC={}
ampbar_Data={}
ampbar_MC={}
normA={}
normAbar={}
normalisationCrossTerms={}
print('INFO: Loading amplitudes...')
for decay in ['b2dk_LL', 'b2dk_DD']:
    for charge in ['p', 'm']:
        #amp_Data[decay+'_'+charge], ampbar_Data[decay+'_'+charge] = getAmp(decay, charge, index=index)
        amp_MC[decay+'_'+charge], ampbar_MC[decay+'_'+charge] = np.load(mc_path + '/Int_%s_%s_amp.npy'%(decay, charge)), np.load(mc_path + '/Int_%s_%s_ampbar.npy'%(decay, charge))

INFO: Loading amplitudes...


In [5]:
from core import Normalisation

In [6]:
norm_dk = Normalisation(amp_MC=amp_MC,ampbar_MC=ampbar_MC,name='b2dk_DD_m')

norm_dk.initialise()


Initialising normalisation for decay: b2dk_DD_m
Normalisation terms:
 |A|^2: tf.Tensor(1.2510769706875426, shape=(), dtype=float64) 
 |Abar|^2: tf.Tensor(1.2526407190700244, shape=(), dtype=float64) 
 |A||Abar|cos(phase): tf.Tensor(0.13513565879072897, shape=(), dtype=float64) 
 |A||Abar| sin(phase): tf.Tensor(0.00022638549837434126, shape=(), dtype=float64)


In [7]:
norm_dk.initialise_misid()

Initialising misid normalisation for decay: b2dpi_DD_m


ValueError: Misid amplitude not found

In [8]:
norm_dk.Integrated_BacTerms()

[<tf.Tensor: shape=(), dtype=float64, numpy=1.2510769706875426>,
 <tf.Tensor: shape=(), dtype=float64, numpy=1.2526407190700244>]

In [9]:
norm_dk.Integrated_crossTerms()

[<tf.Tensor: shape=(), dtype=float64, numpy=0.13513565879072897>,
 <tf.Tensor: shape=(), dtype=float64, numpy=0.00022638549837434126>]

In [10]:
x=(-0.1, -0.01, 0.1, 0.06)

In [11]:
norm_dk.Integrated_dk(-1, x)

<tf.Tensor: shape=(), dtype=float64, numpy=1.2951671830586995>